In [1]:
%load_ext autoreload
%autoreload 2

from specxplore import importing2 as importing
import matchms
from matchms.importing import load_from_mgf
from matchms.exporting import save_as_mgf
from ms2query.run_ms2query import download_zenodo_files, run_complete_folder
from ms2query.ms2library import create_library_object_from_one_dir
import os
import pandas as pd
import numpy as np

/opt/homebrew/Caskroom/miniconda/base/envs/ms2query/lib/python3.8/site-packages/PubChemPy-1.0.4-py3.8.egg/pubchempy.py:563: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/opt/homebrew/Caskroom/miniconda/base/envs/ms2query/lib/python3.8/site-packages/PubChemPy-1.0.4-py3.8.egg/pubchempy.py:563: SyntaxWarning: "is not" with a literal. Did you mean "!="?


In [2]:
models_and_library_folder_pos = "data_and_output/models/ms2query_models_and_library_pos"
results_folder = os.path.join("data_and_output/npl_out")
experimental_mgf_filename = "data_and_output/npl_raw/GNPS-NIH-NATURALPRODUCTSLIBRARY.mgf"
standard_mgf_filename = "data_and_output/npl_raw/phophe_pos.mgf"


processed_experimental_spectra_filename = "data_and_output/npl_processed/spectra_experimental.mgf"
processed_standard_spectra_filename = "data_and_output/npl_processed/spectra_standards.mgf"
csv_experimental_filename = "data_and_output/npl_out/spectra_experimental.csv"
csv_standards_filename = "data_and_output/npl_out/spectra_standards.csv"

# Process Raw Data


In [3]:
spectra_experimental = list(load_from_mgf(experimental_mgf_filename))
spectra_experimental = importing.clean_spectra(spectra_experimental)
save_as_mgf(spectra_experimental, processed_experimental_spectra_filename)


spectra_standards = list(load_from_mgf(standard_mgf_filename))
spectra_standards = importing.clean_spectra(spectra_standards)
save_as_mgf(spectra_standards, processed_standard_spectra_filename)

[13:52:03] WARNING: Omitted undefined stereo

[13:52:03] WARNING: Omitted undefined stereo

[13:52:03] WARNING: Omitted undefined stereo

[13:52:03] WARNING: Omitted undefined stereo

[13:52:03] WARNING: Omitted undefined stereo

[13:52:03] WARNING: Omitted undefined stereo

[13:52:03] WARNING: Omitted undefined stereo

[13:52:03] WARNING: Omitted undefined stereo

[13:52:03] WARNING: Omitted undefined stereo

[13:52:03] WARNING: Omitted undefined stereo

[13:52:03] WARNING: Omitted undefined stereo

[13:52:03] WARNING: Omitted undefined stereo

[13:52:03] WARNING: Omitted undefined stereo

[13:52:03] WARNING: Omitted undefined stereo

[13:52:03] WARNING: Omitted undefined stereo

[13:52:03] WARNING: Omitted undefined stereo

[13:52:03] WARNING: Omitted undefined stereo

[13:52:03] WARNING: Omitted undefined stereo

[13:52:03] WARNING: Omitted undefined stereo

[13:52:03] WARNING: Omitted undefined stereo

[13:52:03] WARNING: Omitted undefined stereo

[13:52:03] WARNING: Omitted undefi

In [27]:
spectra_experimental[0].get("precursor_mz")

408.156

In [26]:
for spec in spectra_standards + spectra_experimental:
    if not isinstance(spec, matchms.Spectrum):
        print("WTF type")
    if (spec.peaks.mz.size) <= 0:
        print("WTF mz")

# Run Classification using ms2query

In [21]:
# WARNING
# THIS DOES WORK, BUT AUTOMATICALLY RUNS BOTH STANDARDS AND EXPERIMENTAL SPECTRA IF IN SAME FOLDER
# UPDATE TO MS2QUERY LATEST VERSION TO ALLOW SINGLE FILE INPUT AND OUTPUT
# ONCE UPDATED, FILENAMES CAN ALSO BE SPECIFIED....
if False:
    run_complete_folder(ms2library, "public_libraries/gnps_npl", results_folder=results_folder)
    ms2library = create_library_object_from_one_dir(models_and_library_folder_pos)
    run_complete_folder(ms2library, "data_import_testing/phophe-library", results_folder=results_folder)


# Merge Classes Tables and Spectra lists

In [5]:
classes_experimental = pd.read_csv(csv_experimental_filename)
classes_experimental = importing.expand_ms2query_results_table(classes_experimental, len(spectra_experimental))
classes_standards = pd.read_csv(csv_standards_filename)
classes_standards = importing.expand_ms2query_results_table(classes_standards, len(spectra_standards))

In [6]:
classes_experimental["is_standard"] = False
classes_experimental["exp_metadata"] = None
classes_standards["is_standard"] = True
classes_standards["standard_metadata"] = None
all_class_table = pd.merge(classes_experimental, classes_standards, how = 'outer').reset_index()
all_class_table["specxplore_id"] = all_class_table.index
all_class_table

,index,query_spectrum_nr,ms2query_model_prediction,precursor_mz_difference,precursor_mz_query_spectrum,precursor_mz_analog,inchikey,spectrum_ids,analog_compound_name,retention_time,...,cf_subclass,cf_direct_parent,npc_class_results,npc_superclass_results,npc_pathway_results,source_spectrum_mgf_iloc,is_standard,exp_metadata,standard_metadata,specxplore_id
0,0,1,0.9284,0.0000,408.1560,408.156,XTJNPXYDTGJZSA,11275.0,"(2R)-2-[3-[(10aS)-1,3-dioxo-10,10a-dihydro-5H-...",NaN,...,Hybrid peptides,Hybrid peptides,NaN,NaN,Amino acids and Peptides,0,False,None,None,0
1,1,2,0.8834,0.0000,277.0980,277.098,VOYWJNWCKFCMPN,11276.0,"(2S)-3-benzoyl-2,7b-dihydro-1aH-oxireno[2,3-c]...",NaN,...,Hydroquinolines,Hydroquinolines,NaN,NaN,NaN,1,False,None,None,1
2,2,3,0.8556,0.0000,269.0810,269.081,IRZVHDLBAYNPCT,11277.0,5-hydroxy-7-methoxy-2-phenylchromen-4-one,NaN,...,O-methylated flavonoids,7-O-methylated flavonoids,Flavones,Flavonoids,Shikimates and Phenylpropanoids,2,False,None,None,2
3,3,4,0.9379,0.0000,492.1770,492.177,OPWCHZIQXUKNMP,11278.0,"N-[2-[7-methoxy-6-[(Z)-(2,4,6-trioxo-1-prop-2-...",NaN,...,NaN,Benzodioxoles,NaN,NaN,NaN,3,False,None,None,3
4,4,5,0.8533,0.0000,481.2450,481.245,GTBYYVAKXYVRHX,11279.0,(2S)-2-[[(2S)-3-methyl-2-[[(2S)-3-oxo-2-propan...,NaN,...,"Amino acids, peptides, and analogues",Dipeptides,NaN,Oligopeptides,Amino acids and Peptides,4,False,None,None,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1300,1300,34,0.9387,0.0002,581.1858,581.186,HXTFHSYLYXVTHC,13249.0,"5-hydroxy-2-(4-hydroxyphenyl)-7-[(2S,3R,4S,5S,...",NaN,...,Flavonoid glycosides,Flavonoid-7-O-glycosides,Flavanones,Flavonoids,Shikimates and Phenylpropanoids,33,True,NaN,NaN,1300
1301,1301,35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,34,True,NaN,NaN,1301
1302,1302,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,35,True,NaN,NaN,1302
1303,1303,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,36,True,NaN,NaN,1303


In [7]:
all_spectra = spectra_experimental + spectra_standards # list addition

In [32]:
if False:
    scores_s2v = importing.compute_similarities_s2v(all_spectra, models_and_library_folder_pos)
    scores_cos = importing.compute_similarities_cosine(all_spectra, cosine_type="ModifiedCosine")
    scores_ms2ds = importing.compute_similarities_ms2ds(all_spectra, models_and_library_folder_pos)
    np.save(os.path.join("data_and_output", "npl_out", "ms2ds.npy"), scores_ms2ds, allow_pickle=False)
    np.save(os.path.join("data_and_output", "npl_out", "s2v.npy"), scores_s2v, allow_pickle=False)
    np.save(os.path.join("data_and_output", "npl_out", "cos.npy"), scores_cos, allow_pickle=False)

Calculating vectors of reference spectrums:   0%|          | 0/1305 [00:00<?, ?it/s]

1/1 [==============================] - 0s 12ms/step


Calculating vectors of reference spectrums:   0%|          | 2/1305 [00:00<01:11, 18.32it/s]

1/1 [==============================] - 0s 12ms/step


Calculating vectors of reference spectrums:   0%|          | 6/1305 [00:00<00:47, 27.34it/s]

...

Calculating vectors of reference spectrums:  99%|█████████▉| 1297/1305 [00:41<00:00, 31.84it/s]

1/1 [==============================] - 0s 11ms/step


Calculating vectors of reference spectrums: 100%|█████████▉| 1301/1305 [00:41<00:00, 32.15it/s]

1/1 [==============================] - 0s 11ms/step


Calculating vectors of reference spectrums: 100%|██████████| 1305/1305 [00:41<00:00, 31.08it/s]


In [9]:
scores_s2v = np.load(os.path.join("data_and_output", "npl_out", "s2v.npy"))
scores_ms2ds = np.load(os.path.join("data_and_output", "npl_out", "ms2ds.npy"))
scores_cos = np.load(os.path.join("data_and_output", "npl_out", "cos.npy"))

# Run Kmedoid and tsne

In [10]:
k_values = [10,20,30,40,50,75,100, 125, 150, 175, 200,250, 300, 400,500]
random_seeds = [int(np.random.randint(0,1000)) for _ in k_values]
similarities = scores_ms2ds
distances = importing.convert_similarity_to_distance(similarities)
kmedoid_list = importing.run_kmedoid_grid(distances, k_values, random_seeds)
importing.render_kmedoid_fitting_results_in_browser(kmedoid_list)


In [11]:
perplexity_values = [x for x in range(5, 55, 5)]
random_seeds = [int(np.random.randint(0,1000)) for _ in perplexity_values]
tsne_list_pos = importing.run_tsne_grid(distances, perplexity_values, random_seeds)
importing.render_tsne_fitting_results_in_browser(tsne_list_pos)

# Merge into specxplore object

In [12]:
import specxplore.specxplore_data
from specxplore.specxplore_data import specxplore_data, Spectrum

is_standard = np.array(all_class_table["is_standard"])

spec_classes = all_class_table[['cf_kingdom', 'cf_superclass', 'cf_class', 'cf_subclass',
       'cf_direct_parent', 'npc_class_results', 'npc_superclass_results',
       'npc_pathway_results']]

mz = [spec.get("precursor_mz") for spec in all_spectra]

# KmedoidGridEntry(k, cluster_assignments, score, random_states[idx])
kclass_table = pd.DataFrame()
for elem in kmedoid_list:
    kclass_table[elem.k] = elem.cluster_assignments


classes = pd.concat([spec_classes, kclass_table], axis=1)


In [13]:

tsnedf = pd.DataFrame({"x" : tsne_list_pos[7].x_coordinates, "y" : tsne_list_pos[7].y_coordinates})
specxplore_id = np.array(all_class_table["specxplore_id"])

npl_specxplore = specxplore_data(
  scores_ms2ds,scores_s2v, scores_cos, tsnedf,  classes,
  is_standard, all_spectra, mz, specxplore_id)

In [14]:
import pickle
with open('data_and_output/npl_out/npl_specxplore.pickle', 'wb') as file:
  pickle.dump(npl_specxplore, file)